## Imports and Data Download

In [2]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2023-8-01", end="2023-9-24", interval='15m')
dataF.iloc[-1:,:]
dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


## Defining Signal Function

> Do the last few candles have engulfing patterns?

In [3]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if (open>close and previous_open<previous_close and close<previous_open and open>=previous_close):
        return 1    # Selling signal
    
    # Bullish Pattern
    elif (open<close and previous_open>previous_close and close>previous_open and open<=previous_close):
        return 2    # Buying signal
    
    # No Clear Pattern
    else:
        return 0
    
signal = []
signal.append(0)
for i in range(1, len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))

#signal_generator(data)
dataF["signal"] = signal

In [4]:
# Print Statements

#dataF.signal.value_counts()
dataF.iloc[:, :]

,Open,High,Low,Close,Adj Close,Volume,signal
Datetime,,,,,,,
2023-08-01 00:00:00,1.100473,1.100473,1.100231,1.100231,1.100231,0,0
2023-08-01 00:15:00,1.100231,1.100352,1.100110,1.100110,1.100110,0,0
2023-08-01 00:30:00,1.100110,1.100110,1.099989,1.099989,1.099989,0,0
2023-08-01 00:45:00,1.099989,1.100110,1.099989,1.099989,1.099989,0,0
2023-08-01 01:00:00,1.099989,1.100473,1.099989,1.099989,1.099989,0,0
...,...,...,...,...,...,...,...
2023-09-22 21:15:00,1.064963,1.064963,1.064623,1.064623,1.064623,0,0
2023-09-22 21:30:00,1.064623,1.064849,1.064623,1.064849,1.064849,0,0
2023-09-22 21:45:00,1.064736,1.064849,1.064736,1.064849,1.064849,0,0


## Market Connection and Trade Execution

In [5]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleCollector, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails


In [6]:
from config import access_token, accountID
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)

True
True
True


In [7]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', "Low"])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1, :])

    # EXECUTING ORDERS
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['Open'].iloc[-2]-dfstream['Close'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1, :])
    print(TPBuy, " ", SLBuy, " ", TPSell, " ", SLSell)
    signal = 2

    # Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

    # Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

## Executing Orders Automatically

In [8]:
# MANUAL
#trading_job()

# SCHEDULED
scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,6,7,16,31,46', start_date='2023-09-26 01:00:00')
scheduler.start()